In [10]:
!pip install geopandas
!pip install folium
!pip install lightgbm
!pip install utm
!pip install geojson 
!pip install haversine
!pip install pyshp
!pip install pyproj 

     |████████████████████████████████| 225kB 2.7MB/s 
  Created wheel for pyshp: filename=pyshp-2.1.2-cp36-none-any.whl size=36216 sha256=b2bea1addf8e2be602a20a40f2c0ebc2d9bb4e3c941b1694967e4642fc837dfb
  Stored in directory: /root/.cache/pip/wheels/96/6c/53/4112475adf3b831da97f083163d0f38ee6daac9c1b13f7afea
Successfully built pyshp


In [ ]:
# 시각화 도구 설치
!pip install plotnine #R 시각화 
!pip install folium #지도 시각화 

# 그래프에서 한글표현을 위해 폰트를 설치합니다.
!apt-get update -qq
!apt-get install fonts-nanum* -qq

In [1]:
import matplotlib.pyplot as plt  # 그래프 그리는 용도
import matplotlib.font_manager as fm  # 폰트 관련 용도
path = '/usr/share/fonts/truetype/nanum/NanumGothicEco.ttf'  # 설치된 나눔글꼴중 원하는 녀석의 전체 경로를 가져오자
font_name = fm.FontProperties(fname=path, size=10).get_name()
print(font_name)
plt.rc('font', family=font_name)

fm._rebuild()

NanumGothic Eco


In [11]:
import pandas as pd
from matplotlib import pyplot as plt 
import seaborn as sns
import numpy as np
import folium 
import geopandas as gpd
import math
from tqdm.notebook import tqdm
import utm
from shapely.geometry import Point, Polygon
from haversine import haversine
from plotnine import *
import shapefile  #the pyshp module : Should install pyshp module.
import pandas as pd
from pyproj import Proj, transform  #Should install pyproj module.

In [7]:
# shape file geojson으로 바꾸기 
import geopandas
myshpfile = geopandas.read_file('/content/drive/My Drive/Project/Seoul/기타 변수/어린이보호구역/TL_NURSERY_STTUS.shp')
myshpfile.to_file('/content/drive/My Drive/Project/Seoul/기타 변수/어린이보호구역/어린이보호구역.geojson', driver='GeoJSON')

#geojson 읽기 
child_loc = gpd.read_file('/content/drive/My Drive/Project/Seoul/기타 변수/어린이보호구역/어린이보호구역.geojson')
child_loc

,NURY_ID,NURY_NM,NURY_TYPE,H_DONG,NURY_ADDR,ADDR_DC,CNFM_DATE,PSNCPA,AREA_,TM_X,TM_Y,geometry
0,126,리원어린이집,가정,중림동,서울특별시 중구 만리재로33길 21,"서울특별시 중구 만리재로33길 21 (만리동1가, LIG서울역리가) 102-102",2012-05-11,19,85,197100.8043,550722.2949,POINT (197100.804 550722.295)
1,127,마주어린이집,가정,중림동,서울특별시 중구 중림로 10,서울특별시 중구 중림로 10 삼성사이버빌리지 112-102,2012-07-25,19,84,196859.6651,550928.7417,POINT (196859.665 550928.742)
2,128,동화어린이집,국공립,동화동,서울특별시 중구 청구로 64,서울특별시 중구 청구로 64 청구e편한세상아파트 106동 1층,2012-12-11,40,250,201473.1047,551156.7612,POINT (201473.105 551156.761)
3,129,꿈나무어린이집,가정,황학동,서울특별시 중구 청계천로 400,서울특별시 중구 청계천로 400 101동 709호,2013-02-20,20,89,201758.0799,552350.1256,POINT (201758.080 552350.126)
4,130,해피맘어린이집,가정,황학동,서울특별시 중구 청계천로 400,서울특별시 중구 청계천로 400 롯데캐슬베네치아 105동 704호,2013-03-04,20,88,202008.9018,552363.7433,POINT (202008.902 552363.743)
...,...,...,...,...,...,...,...,...,...,...,...,...
6235,3957,본동어린이집,민간,구로제2동,서울특별시 구로구 구로동로40길 21-1,서울특별시 구로구 구로동로40길 21-1 (구로동),2004-07-29,86,496,189734.4996,544075.4706,POINT (189734.500 544075.471)
6236,3958,미키몬테소리어린이집,민간,고척제1동,서울특별시 구로구 경인로 390,서울특별시 구로구 경인로 390 벽산블루밍아파트 관리동1층 미키몬테소리어린이집 (고척동),2000-02-01,64,160,188004.2022,544012.0692,POINT (188004.202 544012.069)
6237,3959,우림어린이집,민간,오류제2동,서울특별시 구로구 서해안로 2368,서울특별시 구로구 서해안로 2368 우림필유상가동 (오류동),2006-02-25,39,185,186449.6331,543851.4551,POINT (186449.633 543851.455)
6238,3960,성민어린이집,민간,개봉제1동,서울특별시 구로구 고척로21나길 48-41,서울특별시 구로구 고척로21나길 48-41,2005-02-01,66,639,186038.9523,544950.1244,POINT (186038.952 544950.124)


In [8]:
# 좌표계 확인
#  korea 2000/central belt 2010 (epsg:5186) to wgs84(epsg:4326)
print('child_loc',child_loc.crs)

child_loc epsg:4326


In [12]:
# Change coordinate system
# korea 2000/central belt 2010 (epsg:5186) to wgs84(epsg:4326)
inProj = Proj(init = 'epsg:5186')
outProj= Proj(init = 'epsg:4326')
latitude = []
longitude= []
for idx,row in child_loc.iterrows():
    x,y  = row.TM_X,row.TM_Y  # korea 2000 좌표계
    nx,ny = transform(inProj,outProj,x,y)     # 새로운 좌표계    
    latitude.append(ny)
    longitude.append(nx)
child_loc['latitude'] = latitude
child_loc['longitude']= longitude

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:294: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  projstring = _prepare_from_string(" ".join((projstring, projkwargs)))
/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axi

In [13]:
child_loc = child_loc.drop(['TM_X','TM_Y', 'geometry'], axis=1)
child_loc

,NURY_ID,NURY_NM,NURY_TYPE,H_DONG,NURY_ADDR,ADDR_DC,CNFM_DATE,PSNCPA,AREA_,latitude,longitude
0,126,리원어린이집,가정,중림동,서울특별시 중구 만리재로33길 21,"서울특별시 중구 만리재로33길 21 (만리동1가, LIG서울역리가) 102-102",2012-05-11,19,85,37.556021,126.967189
1,127,마주어린이집,가정,중림동,서울특별시 중구 중림로 10,서울특별시 중구 중림로 10 삼성사이버빌리지 112-102,2012-07-25,19,84,37.557881,126.964459
2,128,동화어린이집,국공립,동화동,서울특별시 중구 청구로 64,서울특별시 중구 청구로 64 청구e편한세상아파트 106동 1층,2012-12-11,40,250,37.559939,127.016673
3,129,꿈나무어린이집,가정,황학동,서울특별시 중구 청계천로 400,서울특별시 중구 청계천로 400 101동 709호,2013-02-20,20,89,37.570691,127.019901
4,130,해피맘어린이집,가정,황학동,서울특별시 중구 청계천로 400,서울특별시 중구 청계천로 400 롯데캐슬베네치아 105동 704호,2013-03-04,20,88,37.570813,127.022740
...,...,...,...,...,...,...,...,...,...,...,...
6235,3957,본동어린이집,민간,구로제2동,서울특별시 구로구 구로동로40길 21-1,서울특별시 구로구 구로동로40길 21-1 (구로동),2004-07-29,86,496,37.496081,126.883914
6236,3958,미키몬테소리어린이집,민간,고척제1동,서울특별시 구로구 경인로 390,서울특별시 구로구 경인로 390 벽산블루밍아파트 관리동1층 미키몬테소리어린이집 (고척동),2000-02-01,64,160,37.495489,126.864348
6237,3959,우림어린이집,민간,오류제2동,서울특별시 구로구 서해안로 2368,서울특별시 구로구 서해안로 2368 우림필유상가동 (오류동),2006-02-25,39,185,37.494020,126.846772
6238,3960,성민어린이집,민간,개봉제1동,서울특별시 구로구 고척로21나길 48-41,서울특별시 구로구 고척로21나길 48-41,2005-02-01,66,639,37.503913,126.842107


## 행정동 Mapping

In [14]:
dong_geo = gpd.read_file('/content/drive/My Drive/Project/Seoul/서울시_행정동경계/hangjeongdong_boundary.geojson')
dong_geo

,행정동코드,시군구명,읍면동명,위도,경도,geometry
0,1111053000,종로구,사직동,37.574108,126.970144,"MULTIPOLYGON (((126.97689 37.57565, 126.97703 ..."
1,1111054000,종로구,삼청동,37.588013,126.981114,"MULTIPOLYGON (((126.98269 37.59507, 126.98337 ..."
2,1111055000,종로구,부암동,37.596699,126.962557,"MULTIPOLYGON (((126.97585 37.59656, 126.97359 ..."
3,1111056000,종로구,평창동,37.613965,126.969274,"MULTIPOLYGON (((126.97507 37.63139, 126.97649 ..."
4,1111057000,종로구,무악동,37.577741,126.958989,"MULTIPOLYGON (((126.96067 37.58080, 126.96281 ..."
...,...,...,...,...,...,...
420,1174070000,강동구,둔촌제2동,37.531514,127.147414,"MULTIPOLYGON (((127.15355 37.53400, 127.15357 ..."
421,1174057000,강동구,암사제1동,37.551855,127.134586,"MULTIPOLYGON (((127.14447 37.55477, 127.14319 ..."
422,1174061000,강동구,천호제2동,37.544691,127.122275,"MULTIPOLYGON (((127.13458 37.54712, 127.13115 ..."
423,1174068500,강동구,길동,37.539725,127.145907,"MULTIPOLYGON (((127.14857 37.54578, 127.14871 ..."


In [34]:
# creating a geometry column 
geometry = [Point(xy) for xy in zip(child_loc['longitude'], child_loc['latitude'])]
# Coordinate reference system : WGS84
crs = {'init': 'epsg:4326'}
# Creating a Geographic data frame 
gdf_child_loc = gpd.GeoDataFrame(child_loc, crs=crs, geometry=geometry)

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [36]:
# 행정동 포함여부 확인 
gdf_child_loc['행정동코드'] = np.zeros(len(gdf_child_loc))
gdf_child_loc['행정동'] = np.zeros(len(gdf_child_loc))

for i in tqdm(gdf_child_loc.index):
    for j in dong_geo.index:
        if gdf_child_loc.geometry[i].within(dong_geo.geometry[j]):
            gdf_child_loc['행정동'][i] = dong_geo.읍면동명[j]
            gdf_child_loc['행정동코드'][i] = dong_geo.행정동코드[j]
            
gdf_child_loc['행정동코드'] = gdf_child_loc['행정동코드'].astype(int)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [37]:
gdf_child_loc

,NURY_ID,NURY_NM,NURY_TYPE,H_DONG,NURY_ADDR,ADDR_DC,CNFM_DATE,PSNCPA,AREA_,latitude,longitude,geometry,행정동코드,행정동
0,126,리원어린이집,가정,중림동,서울특별시 중구 만리재로33길 21,"서울특별시 중구 만리재로33길 21 (만리동1가, LIG서울역리가) 102-102",2012-05-11,19,85,37.556021,126.967189,POINT (126.96719 37.55602),1114068000,중림동
1,127,마주어린이집,가정,중림동,서울특별시 중구 중림로 10,서울특별시 중구 중림로 10 삼성사이버빌리지 112-102,2012-07-25,19,84,37.557881,126.964459,POINT (126.96446 37.55788),1114068000,중림동
2,128,동화어린이집,국공립,동화동,서울특별시 중구 청구로 64,서울특별시 중구 청구로 64 청구e편한세상아파트 106동 1층,2012-12-11,40,250,37.559939,127.016673,POINT (127.01667 37.55994),1114066500,동화동
3,129,꿈나무어린이집,가정,황학동,서울특별시 중구 청계천로 400,서울특별시 중구 청계천로 400 101동 709호,2013-02-20,20,89,37.570691,127.019901,POINT (127.01990 37.57069),1114067000,황학동
4,130,해피맘어린이집,가정,황학동,서울특별시 중구 청계천로 400,서울특별시 중구 청계천로 400 롯데캐슬베네치아 105동 704호,2013-03-04,20,88,37.570813,127.022740,POINT (127.02274 37.57081),1114067000,황학동
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6235,3957,본동어린이집,민간,구로제2동,서울특별시 구로구 구로동로40길 21-1,서울특별시 구로구 구로동로40길 21-1 (구로동),2004-07-29,86,496,37.496081,126.883914,POINT (126.88391 37.49608),1153053000,구로제2동
6236,3958,미키몬테소리어린이집,민간,고척제1동,서울특별시 구로구 경인로 390,서울특별시 구로구 경인로 390 벽산블루밍아파트 관리동1층 미키몬테소리어린이집 (고척동),2000-02-01,64,160,37.495489,126.864348,POINT (126.86435 37.49549),1153072000,고척제1동
6237,3959,우림어린이집,민간,오류제2동,서울특별시 구로구 서해안로 2368,서울특별시 구로구 서해안로 2368 우림필유상가동 (오류동),2006-02-25,39,185,37.494020,126.846772,POINT (126.84677 37.49402),1153078000,오류제2동
6238,3960,성민어린이집,민간,개봉제1동,서울특별시 구로구 고척로21나길 48-41,서울특별시 구로구 고척로21나길 48-41,2005-02-01,66,639,37.503913,126.842107,POINT (126.84211 37.50391),1153074000,개봉제1동


In [38]:
gdf_child_loc = gdf_child_loc.sort_values(by='NURY_ID').reset_index().drop('index',axis=1)

In [39]:
gdf_child_loc = gdf_child_loc[['행정동코드', '행정동', 'NURY_NM', 'NURY_TYPE', 'NURY_ADDR', 'ADDR_DC',
       'CNFM_DATE', 'PSNCPA', 'AREA_', 'latitude', 'longitude', 'geometry']]
gdf_child_loc

,행정동코드,행정동,NURY_NM,NURY_TYPE,NURY_ADDR,ADDR_DC,CNFM_DATE,PSNCPA,AREA_,latitude,longitude,geometry
0,1111063000,종로5.6가동,이화어린이집,민간,서울특별시 종로구 율곡로20길 3,서울특별시 종로구 율곡로20길 3 (충신동),2004-05-03,64,266,37.574928,127.005126,POINT (127.00513 37.57493)
1,1111054000,삼청동,삼청어린이집,국공립,서울특별시 종로구 삼청로7길 26,서울특별시 종로구 삼청로7길 26 (팔판동),1999-09-01,32,95,37.583998,126.980473,POINT (126.98047 37.58400)
2,1111068000,창신제2동,창신어린이집,사회복지법인,서울특별시 종로구 창신5길 22,서울특별시 종로구 창신5길 22 (창신동),1978-12-01,77,444,37.573626,127.012002,POINT (127.01200 37.57363)
3,1111069000,창신제3동,아동회관어린이집,국공립,서울특별시 종로구 지봉로13길 14,서울특별시 종로구 지봉로13길 14 아동회관어린이집,1988-10-01,96,325,37.578911,127.014515,POINT (127.01452 37.57891)
4,1111055000,부암동,부암어린이집,국공립,서울특별시 종로구 세검정로6다길 10-7,서울특별시 종로구 세검정로6다길 10-7,1998-07-29,89,307,37.601515,126.963077,POINT (126.96308 37.60152)
...,...,...,...,...,...,...,...,...,...,...,...,...
6235,1174066000,성내제3동,구립색종이어린이집,국공립,서울특별시 강동구 강동대로53길 66,"서울특별시 강동구 강동대로53길 66 102호(성내동, 한솔애리즈5차)",2017-02-15,20,104,37.525730,127.132673,POINT (127.13267 37.52573)
6236,1174054000,명일제2동,강동경희대병원어린이집,직장,서울특별시 강동구 동남로 881,서울특별시 강동구 동남로 881 5층 서희스타힐스(명일동),2017-03-02,39,183,37.552228,127.155755,POINT (127.15576 37.55223)
6237,1174055000,고덕제1동,애플어린이집,가정,서울특별시 강동구 아리수로50길 50,서울특별시 강동구 아리수로50길 50 124동 204호,2017-03-27,20,99,37.559794,127.148076,POINT (127.14808 37.55979)
6238,1174064000,성내제1동,구립올림픽파크꿈별어린이집,국공립,서울특별시 강동구 성내로 80,서울특별시 강동구 성내로 80,2017-05-25,24,109,37.526745,127.128286,POINT (127.12829 37.52674)


In [40]:
gdf_child_loc.to_file('/content/drive/My Drive/Project/Seoul/기타 변수/어린이보호구역/어린이보호구역_좌표변환.geojson', driver='GeoJSON')

In [41]:
gdf_child_loc.to_csv('/content/drive/My Drive/Project/Seoul/기타 변수/어린이보호구역/어린이보호구역_좌표변환.csv', index=False)

In [53]:
# 어린이집 아닌 어린이 보호구역 필터링 
df_merge = gdf_child_loc.copy()
mask1 = df_merge['NURY_NM'].str.contains('어린이집')  

df_kinder = df_merge[~mask1].reset_index().drop(['index'], axis=1)
df_kinder # 대부분이 어린이집 나머지는 방과후 교실 

,행정동코드,행정동,NURY_NM,NURY_TYPE,NURY_ADDR,ADDR_DC,CNFM_DATE,PSNCPA,AREA_,latitude,longitude,geometry
0,1111058000,교남동,다솔 방과후교실,민간,서울특별시 종로구 통일로8길 16,서울특별시 종로구 통일로8길 16 (송월동),2006-03-13,39,444,37.569280,126.965478,POINT (126.96548 37.56928)
1,1117057000,원효로제2동,선인방과후교실,법인·단체등,서울특별시 용산구 효창원로8길 17,서울특별시 용산구 효창원로8길 17 선인중앙교회 (원효로4가),2006-04-06,40,277,37.533897,126.952131,POINT (126.95213 37.53390)
2,1129071500,월곡제1동,월곡청소년센터 방과후어린이교실,법인·단체등,서울 성북구 월곡1동 산2-1,서울 성북구 월곡1동 산2-1,1998-10-01,40,99,37.609585,127.041507,POINT (127.04151 37.60958)
3,1129076000,장위제1동,장위신협방과후교실,법인·단체등,서울특별시 성북구 장위로 60,서울특별시 성북구 장위로 60 장위신협 방과후교실 (장위동),2000-08-30,20,132,37.613770,127.043231,POINT (127.04323 37.61377)
4,1129059000,돈암제2동,꿈보,가정,서울특별시 성북구 북악산로 844,"서울특별시 성북구 북악산로 844 103동 103호(돈암동, 돈암이수브라운스톤아파트)",2005-01-04,17,59,37.597299,127.018668,POINT (127.01867 37.59730)
5,1130559500,번1동,주영광방과후교실,민간,서울특별시 강북구 한천로 1004,서울특별시 강북구 한천로 1004 (번동),2007-03-07,39,143,37.638319,127.030000,POINT (127.03000 37.63832)
6,1132071000,방학제3동,도봉구육아종합지원센터 방과후교실,국공립,서울특별시 도봉구 방학로12길 28,서울특별시 도봉구 방학로12길 28 (방학동),2006-04-10,40,110,37.663226,127.030334,POINT (127.03033 37.66323)
7,1138056000,구산동,구산방과후교실,법인·단체등,서울특별시 은평구 연서로11길 32,서울특별시 은평구 연서로11길 32 구산교회방과후교실 (구산동),2005-02-01,20,109,37.609293,126.911982,POINT (126.91198 37.60929)
8,1138051000,녹번동,성암교회방과후교실,법인·단체등,서울특별시 은평구 서오릉로4길 6,서울특별시 은평구 서오릉로4길 6 (녹번동),2004-03-02,40,366,37.604732,126.925570,POINT (126.92557 37.60473)
9,1138063100,신사제1동,은평교회방과후교실,법인·단체등,서울특별시 은평구 은평로3길 10,서울특별시 은평구 은평로3길 10 은평성결교회 (역촌동),2002-12-20,20,144,37.599680,126.915131,POINT (126.91513 37.59968)


## 행정동 별 어린이보호구역 개수

In [54]:
df_num = gdf_child_loc.groupby(by=['행정동코드','행정동'], as_index=False).count()[['행정동','행정동코드','NURY_NM']]
df_num.columns = ['행정동','행정동코드','어린이보호구역개수']

In [56]:
df_num

,행정동,행정동코드,어린이보호구역개수
0,청운효자동,1111051500,6
1,사직동,1111053000,11
2,삼청동,1111054000,1
3,부암동,1111055000,3
4,평창동,1111056000,4
...,...,...,...
419,성내제2동,1174065000,14
420,성내제3동,1174066000,13
421,길동,1174068500,28
422,둔촌제1동,1174069000,5


In [57]:
# 결측치 0으로 채움 
df_num = dong_geo.merge(df_num, how ='outer', on='행정동코드').drop(['행정동','위도', '경도','geometry' ],axis=1).fillna(0)
df_num

,행정동코드,시군구명,읍면동명,어린이보호구역개수
0,1111053000,종로구,사직동,11.0
1,1111054000,종로구,삼청동,1.0
2,1111055000,종로구,부암동,3.0
3,1111056000,종로구,평창동,4.0
4,1111057000,종로구,무악동,4.0
...,...,...,...,...
420,1174070000,강동구,둔촌제2동,19.0
421,1174057000,강동구,암사제1동,22.0
422,1174061000,강동구,천호제2동,9.0
423,1174068500,강동구,길동,28.0


In [58]:
df_num.to_csv('/content/drive/My Drive/Project/Seoul/가공데이터/행정동별_어린이보호구역개수.csv')